In [2]:
import pandas as pd
from tqdm import tqdm
from pandarallel import pandarallel

In [ ]:
df_recount3 = pd.read_csv("ftp-data/recount3.bed", sep="\t", names=["Chrom", "Start", "End", "Features", "Score", "Strand"])
df_gencode_introns = pd.read_csv("gencode_v44_introns.csv")
df_recount3

In [ ]:
df_featuers_split = df_recount3["Features"].str.split(";")
df_featuers_split

In [ ]:
df_recount3["GTExv2"] = df_featuers_split.str[0].str.split("=").str[1]

In [ ]:
df_recount3["TCGAv2"] = df_featuers_split.str[1].str.split("=").str[1]

In [ ]:
df_recount3["SRAv3h"] = df_featuers_split.str[2].str.split("=").str[1]

In [ ]:
df_recount3["Splice_site"] = df_featuers_split.str[3]
df_recount3

In [ ]:
def split_features(row):
    # Split the "Features" column by semicolon
    # print("OK")
    # print(row)
    features_split = row.split(";")
    # print(features_split)
    # Extract the desired values from the split columns and assign them to new columns
    GTExv2 = features_split[0].split("=")[1]
    TCGAv2 = features_split[1].split("=")[1]
    SRAv3h = features_split[2].split("=")[1]
    Splice_site = features_split[3]
    return pd.Series([GTExv2, TCGAv2, SRAv3h, Splice_site])

In [6]:
# df_recount3.to_csv("gencode_v44_introns_modified.csv", index=False)

# If we run from here
df_recount3 = pd.read_csv("gencode_v44_introns_modified.csv")
df_gencode_introns = pd.read_csv("gencode_v44_introns.csv")

# We try to match them

In [7]:
df_recount3.drop(columns=["Features"], inplace=True)

In [8]:
# pandarallel.initialize(progress_bar=True)
df_gencode_introns_dict = df_gencode_introns.to_dict()  # Change it to dict

In [9]:
df_recount3_test = df_recount3.copy()
df_recount3_test

,Chrom,Start,End,Score,Strand,GTExv2,TCGAv2,SRAv3h,Splice_site
0,chr1,10017,10106,26,-,0:0,0:0,19:26,GT:AG
1,chr1,10017,10178,40,-,0:0,0:0,24:40,GT:AG
2,chr1,10023,10106,98,-,0:0,0:0,60:98,GT:AG
3,chr1,10023,10178,123,-,0:0,0:0,72:123,GT:AG
4,chr1,10029,10106,307,-,0:0,0:0,150:307,GT:AG
...,...,...,...,...,...,...,...,...,...
16350176,chrY_KI270740v1_random,34728,36982,36,-,0:0,0:0,18:36,GT:AG
16350177,chrY_KI270740v1_random,34853,36982,37,-,0:0,0:0,21:37,GT:AG
16350178,chrY_KI270740v1_random,34853,37107,29,-,2:2,0:0,22:27,GT:AG
16350179,chrY_KI270740v1_random,35132,35257,39,+,0:0,0:0,28:39,GT:AG


In [10]:
df_recount3_test.set_index(["Start", "End", "Strand"], inplace=True)

In [11]:
df_recount3_test.sort_index(inplace=True)

In [ ]:
# df_recount3_test.loc[(10029, 10106, "-"), :]

In [21]:
def match_recount(row):
    # print(row)
    # start = row.intron_start
    # end = row.intron_ends
    # strand = row.strand
    try:
        df_matched = df_recount3_test.loc[(row.intron_start, row.intron_ends, row.strand), :]
    except:
        df_matched = "UNMATCHED"
    # print(df_matched)

In [25]:
pandarallel.initialize(nb_workers=8, progress_bar=True)

trail_run_gencode = df_gencode_introns.copy().head(20)
trail_run_gencode

In [27]:
trail_run_gencode.parallel_apply(match_recount, axis=1)
trail_run_gencode

In [19]:
trail_run_gencode

,gene_id,transcript_id,intron_id,intron_start,intron_ends,strand,prev_exon_id,next_exon_id
0,ENSG00000290825.1,ENST00000456328.2,0,12227,12612,+,ENSE00002234944.1,ENSE00003582793.1
1,ENSG00000290825.1,ENST00000456328.2,1,12721,13220,+,ENSE00003582793.1,ENSE00002312635.1
2,ENSG00000223972.6,ENST00000450305.2,0,12057,12178,+,ENSE00001948541.1,ENSE00001671638.2
3,ENSG00000223972.6,ENST00000450305.2,1,12227,12612,+,ENSE00001671638.2,ENSE00001758273.2
4,ENSG00000223972.6,ENST00000450305.2,2,12697,12974,+,ENSE00001758273.2,ENSE00001799933.2
...,...,...,...,...,...,...,...,...
4995,ENSG00000142606.16,ENST00000378412.8,0,2591089,2591556,-,ENSE00001957118.1,ENSE00003649471.1
4996,ENSG00000142606.16,ENST00000378412.8,1,2591633,2591931,-,ENSE00003649471.1,ENSE00003599396.1
4997,ENSG00000142606.16,ENST00000378412.8,2,2592027,2592654,-,ENSE00003599396.1,ENSE00003481724.1
4998,ENSG00000142606.16,ENST00000378412.8,3,2592720,2592832,-,ENSE00003481724.1,ENSE00003641328.1
